# How to use ffsim's Qiskit Sampler primitive

In this guide, we show how to use [FfsimSampler](../api/ffsim.qiskit.rst#ffsim.qiskit.FfsimSampler), ffsim's implementation of the Qiskit [Sampler primitive](https://docs.quantum.ibm.com/api/qiskit/qiskit.primitives.BaseSamplerV2), to sample from quantum circuits built from fermionic gates.

## Example of using FfsimSampler

First, let's create an example circuit using gates from the [ffsim.qiskit](../api/ffsim.qiskit.rst) module.


In [1]:
import numpy as np
from qiskit.circuit import QuantumCircuit, QuantumRegister

import ffsim

# Let's use 8 spatial orbitals with 4 alpha electrons and 4 beta electrons.
norb = 8
nelec = (4, 4)
n_alpha, n_beta = nelec

# Generate some random data
rng = np.random.default_rng(12345)
orbital_rotation = ffsim.random.random_unitary(norb, seed=rng)
diag_coulomb_mat = ffsim.random.random_real_symmetric_matrix(norb, seed=rng)

# Create an example circuit
qubits = QuantumRegister(2 * norb)
circuit = QuantumCircuit(qubits)
circuit.append(
    ffsim.qiskit.PrepareSlaterDeterminantJW(
        norb,
        occupied_orbitals=[range(n_alpha), range(n_beta)],
        orbital_rotation=orbital_rotation,
    ),
    qubits,
)
circuit.append(
    ffsim.qiskit.DiagCoulombEvolutionJW(norb, diag_coulomb_mat, time=1.0), qubits
)
circuit.append(ffsim.qiskit.OrbitalRotationJW(norb, orbital_rotation.T.conj()), qubits)
circuit.measure_all()

Next, we initialize the ffsim sampler and use it to sample 10,000 shots from our circuit. The input to the sampler is a list of [primitive unified blocs](https://docs.quantum.ibm.com/api/qiskit/primitives#overview-of-samplerv2), or PUBs. In the cell output we display only the top 10 most commonly encountered bitstrings.


In [2]:
# Initialize ffsim sampler
sampler = ffsim.qiskit.FfsimSampler(default_shots=10_000, seed=rng)

# Form PUB, submit job, retrieve job result, and extract first (and only) PUB result
pub = (circuit,)
job = sampler.run([pub])
result = job.result()
pub_result = result[0]

# Get counts
counts = pub_result.data.meas.get_counts()

# Display the 10 most commonly seen bitstrings and their counts
{k: v for k, v in sorted(counts.items(), key=lambda x: x[1], reverse=True)[:10]}

{'0000111100001111': 210,
 '0000111100101011': 133,
 '0010101100001111': 101,
 '0000111100011101': 70,
 '0010011100101101': 66,
 '0010110100101011': 65,
 '0010110100100111': 57,
 '0001110100001111': 56,
 '0000111100011011': 48,
 '0010101100101101': 47}

And that's it!

## Criteria for circuits that FfsimSampler can sample

FfsimSampler cannot sample from arbitrary Qiskit circuits. It can handle circuits that satisfy the following criteria:

- The circuit contains only the following types of gates:
  - Any gate from the [ffsim.qiskit](../api/ffsim.qiskit.rst) module.
  - [Barrier](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.Barrier#barrier).
  - [Measure](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.Measure#measure).
- The first gate of the circuit is one of the following gates:
  - [ffsim.qiskit.PrepareHartreeFockJW](../api/ffsim.qiskit.rst#ffsim.qiskit.PrepareHartreeFockJW).
  - [ffsim.qiskit.PrepareHartreeFockSpinlessJW](../api/ffsim.qiskit.rst#ffsim.qiskit.PrepareHartreeFockSpinlessJW).
  - [ffsim.qiskit.PrepareSlaterDeterminantJW](../api/ffsim.qiskit.rst#ffsim.qiskit.PrepareSlaterDeterminantJW).
  - [ffsim.qiskit.PrepareSlaterDeterminantSpinlessJW](../api/ffsim.qiskit.rst#ffsim.qiskit.PrepareSlaterDeterminantSpinlessJW).
- The state preparation gates listed in the previous item only appear as the first gate of the circuit.
- All measurements are performed at the end of the circuit. In other words, the circuit does not contain any mid-circuit measurements.

Due to the need to preserve fermionic statistics, some of the supported gates must be applied to consecutive qubits in ascending order.


## Example of sampling from an LUCJ circuit for a nitrogen molecule

The following code cell demonstrates a possible workflow for sampling from an LUCJ circuit for a nitrogen molecule in the 6-31g basis.


In [3]:
import pyscf.data.elements
from pyscf import cc, gto

# Build N2 molecule
mol = gto.Mole()
mol.build(
    atom=[["N", (0, 0, 0)], ["N", (1.0, 0, 0)]],
    basis="6-31g",
    symmetry="Dooh",
)

# Define active space
n_frozen = pyscf.data.elements.chemcore(mol)
active_space = range(n_frozen, mol.nao_nr())

# Get molecular data and Hamiltonian
mol_data = ffsim.MolecularData.from_mole(mol, active_space=active_space)
norb, nelec = mol_data.norb, mol_data.nelec
mol_hamiltonian = mol_data.hamiltonian
print(f"norb = {norb}")
print(f"nelec = {nelec}")

# Get CCSD t2 amplitudes for initializing the ansatz
ccsd = cc.CCSD(
    mol_data.scf,
    frozen=[i for i in range(mol.nao_nr()) if i not in active_space],
)
_, _, t2 = ccsd.kernel()

# Construct LUCJ operator using indices for a square lattice mapping
n_reps = 1
alpha_alpha_indices = [(p, p + 1) for p in range(norb - 1)]
alpha_beta_indices = [(p, p) for p in range(norb)]
ucj_op = ffsim.UCJOperator.from_parameters(
    ffsim.UCJOperator.from_t_amplitudes(t2).to_parameters(),
    norb=norb,
    n_reps=n_reps,
    alpha_alpha_indices=alpha_alpha_indices,
    alpha_beta_indices=alpha_beta_indices,
)

# Construct circuit
qubits = QuantumRegister(2 * norb)
circuit = QuantumCircuit(qubits)
circuit.append(ffsim.qiskit.PrepareHartreeFockJW(norb, nelec), qubits)
circuit.append(ffsim.qiskit.UCJOperatorJW(ucj_op), qubits)
circuit.measure_all()

# Sample 10,000 shots from the circuit using FfsimSampler
sampler = ffsim.qiskit.FfsimSampler(default_shots=10_000, seed=12345)
pub = (circuit,)
job = sampler.run([pub])
result = job.result()
pub_result = result[0]
counts = pub_result.data.meas.get_counts()

# Display the 10 most commonly seen bitstrings and their counts
{k: v for k, v in sorted(counts.items(), key=lambda x: x[1], reverse=True)[:10]}

converged SCF energy = -108.835236570775
norb = 16
nelec = (5, 5)
E(CCSD) = -109.0398256929734  E_corr = -0.2045891221988307


{'00000000000111110000000000011111': 9897,
 '00000000010110110000000000011111': 28,
 '00000000000111110000000001011011': 20,
 '00100000000110110000000000011111': 7,
 '10000000000011110000000000011111': 7,
 '00000000000111110000000000110111': 6,
 '00000000000111110010000000011011': 6,
 '00000000001101110000000000110111': 3,
 '01000000000011110000000000011111': 3,
 '00000000000111111000000000001111': 3}